In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pandas as pd
import numpy as np
from datasurfer.lib_objects.pdf_object import PDFPagesObject
from datasurfer.datautils import xml_valid_df
from pathlib import Path

In [56]:
num_chapter = 7
rng_pages = (493-1, 577)

root = Path(r'C:\95_Programming\70_Language_Related\10_LLM_Translation\20_Integrale_Sicherheit_von_Kraftfahrzeugen')
wdir = root / f'{num_chapter:02d}_ISK_Chapter_{num_chapter:02d}'
wdir.mkdir(exist_ok=True)
fdst = wdir / f'ISK_Chapter_{num_chapter:02d}_Original.xlsx'

In [57]:
pobj = PDFPagesObject(root / 'ISK.pdf', range(*rng_pages))
df = pobj.dataframe

In [58]:

df = df[df.text.str.strip().str.len()>0].reset_index(drop=True)
df['rel_pagenum'] = df.page_num.astype(int) - df.page_num.astype(int).min()


In [59]:
df = df[df.text.str.contains(r'[a-zA-Z]+', regex=True)].reset_index(drop=True)

In [60]:

def remove_title_line(df):
    count = 0
    for i, (text, num) in df[['text', 'rel_pagenum']].iterrows():  
        if num > count:
            count += 1       
        else:        
            yield([text, num])
        
out0 = list(remove_title_line(df))



In [61]:
def split_paragraph(text, threshold=0.8):
    
    lst_text = np.array([txt for txt in text.split('\n') if txt.strip()])
    larr = np.array([len(txt) for txt in lst_text])
    isnewline = larr < (larr.max() * threshold)
    
    out = ''
    for isnl, txt in zip(isnewline, lst_text):
        if isnl:
            out += txt
            yield out
            out = ''
        else:
            out += txt.rstrip('-')
            
    if len(out) > 0:      
        yield out

    


In [62]:

def yield_paragraphs(out0):

    for text, num in out0:
        for txt in split_paragraph(text):
            yield [txt, num]
            
out1 = list(yield_paragraphs(out0))


In [63]:

def merge_paragraphs(out1):
    
    iterator = iter(out1)
    line0, num0 = next(iterator)
    while True:
        try:       
            line1, num1 = next(iterator)
        except StopIteration:
            yield line0, num0
            break
        if num1 > num0:
        
            if line0.strip()[-1] not in '.!?:;':
                line = line0 + ' ' + line1
                yield line, num0
                try:
                    line0, num0 = next(iterator)
                except StopIteration:
                    yield line0, num0
                    break
            else:
                yield line0, num0
                line0 = line1
                num0 = num1            
        else:
            yield line0, num0   
            line0 = line1
            num0 = num1
            
out2 = list(merge_paragraphs(out1))

In [64]:
dfout = pd.DataFrame(out2, columns=['Original', 'Page'])
dfout['Translation'] =  None
dfout['Review'] = 4 

dfout = xml_valid_df(dfout[['Page', 'Original', 'Translation', 'Review']])
dfout.Page = dfout.Page.astype('int') + df.page_num.min() + 1

dfout = dfout[~dfout['Original'].str.isnumeric()]
dfout = dfout[~(dfout['Original'].str.strip().str.len()<2)]
dfout.reset_index(drop=True, inplace=True)

In [65]:
dfout.to_excel(fdst, index=False)

In [1]:
from trans_preproc import extract_chapter_text

In [7]:
num_chapter = 7
rng_pages = range(493-1, 577)
root = Path(r'C:\95_Programming\70_Language_Related\10_LLM_Translation\20_Integrale_Sicherheit_von_Kraftfahrzeugen')
wdir = root / f'{num_chapter:02d}_ISK_Chapter_{num_chapter:02d}'
wdir.mkdir(exist_ok=True)
fdst = wdir / f'ISK_Chapter_{num_chapter:02d}_Original.xlsx'

extract_chapter_text(root/'ISK.pdf', rng_pages).to_excel(fdst, index=False)